In [1]:
import torch
import torchvision.models as models
import os

In [21]:
model=models.mobilenet_v2(pretrained=True)
file_to_save=os.path.join('.','model_weights','mobile_net_v2_no_change.model')
torch.save(model, file_to_save)

In [6]:
loaded_model=torch.load('./model_weights/mobile_net_v2_no_change.model')
loaded_model.eval()

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [2]:
from torchvision import transforms
from PIL import Image

with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [3]:
preprocess=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

In [8]:
def get_label(image,categories=categories,preprocess=preprocess):
    input_tensor=preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) #needed to work with images of batch size 0
    with torch.no_grad():
        output = loaded_model(input_batch)
        
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    top5_prob, top5_catid = torch.topk(probabilities, 5)
    result = {}
    for i in range(top5_prob.size(0)):
        result[categories[top5_catid[i]]] = top5_prob[i].item()
    return result

In [25]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2021-10-19 23:56:50--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.02s   

2021-10-19 23:56:51 (421 KB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [13]:
filename='/home/ryk/Documents/Pet Projects/NLP/image_classification_app/data/dog.jpg'
input_image = Image.open(filename)
get_label(input_image ,categories=categories,preprocess=preprocess)

{'golden retriever': 0.9139328002929688,
 'clumber': 0.01301341038197279,
 'Tibetan mastiff': 0.011807659640908241,
 'Brittany spaniel': 0.010620344430208206,
 'Labrador retriever': 0.00994053389877081}